In [1]:
import sys
import os
import pandas as pd
import numpy as np

from collections import Counter
from bareunpy._tagger import Tagger, Tagged
from bareunpy._tokenizer import Tokenizer, Tokenized
from bareunpy._custom_dict import CustomDict
from bareunpy._custom_dict_client import CustomDictionaryServiceClient
from bareunpy._lang_service_client import BareunLanguageServiceClient

import json
from sys import stdout
from typing import IO, List, Any, Union

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


import bareunpy as brn
from google.protobuf.json_format import MessageToDict
import grpc
from bareunpy._custom_dict import CustomDict
from bareunpy._lang_service_client import BareunLanguageServiceClient, MAX_MESSAGE_LENGTH
from bareun.language_service_pb2 import AnalyzeSyntaxResponse, AnalyzeSyntaxListResponse, Morpheme, Sentence, Token

In [49]:
naver_shopping = pd.read_csv(r'C:\Users\NT550\asac\SIH\SIH\code2\result\naver_shopping_review.csv')
naver_shopping['제품명'].value_counts()

허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]오늘출발               8936
하루벌꿀 아카시아꿀 100회분 꿀스틱 벌꿀 [원산지:국산]오늘출발                             4728
허니바이허니 천연 벌꿀 스틱 30포입 국산 아카시아 5종 답례품 선물 세트 [원산지:국산]오늘출발           3251
국내산 천연벌꿀 찐 허니스틱 100포 스틱꿀 3개월분 아카시아 야생화 밤꿀 혼합형 [원산지:국산]오늘출발       3119
허니바이허니 국산 천연 벌꿀 스틱 선물 세트 30포입 답례품 아카시아 야생화 밤꿀 감로 [원산지:국산]오늘출발    2897
                                                                 ... 
오지퀸비 마누카꿀스틱 MGO150+30개입 [원산지:호주산(오지퀸비)]                             9
꿀벌 백만송이 날개짓 사양꿀 스틱꿀 납품용 1000개 종이박스 [원산지:국산]                         9
호주 BEE2비투 허니스틱 50개 기프트박스 [원산지:호주산(BEE2)]                            5
꿀마늘 280g 마늘꿀 ,마늘꿀절임 스틱꿀마늘 [원산지:국산(경상북도 영천시) 등]무료교환반품                5
호주 BEE2비투 허니스틱 1박스 250개 [원산지:호주산(BEE2)]                             3
Name: 제품명, Length: 147, dtype: int64

In [3]:
# df_review = review for review in naver_shopping['리뷰'] if review == str 
df_review = [review for review in naver_shopping['리뷰'] if type(review) is str]
df_review

['한달사용기재구매먹기 편하고 품질도 좋아요. 받는 손님도 좋아 하시고 구매등급도 따로 있어서 좋아요. 문제 생기면 바로 처리 해주시고 좋아요!',
 '간편하게 하루한포씩먹게되요~냉장이나 냉동으로 해서 시원하게 먹으니 맛잇어요!!밤꿀이 많이 쓸줄알앗는데 쓴건 잘 안먹는편이기도 하구요  생각보다 진하니 먹을만했어요 요즘 면역이 많이 떨어지기도 했고 겸사겸사 구매했는데 마음에들어서 다음에 재구매해야겠어요! 선물용으로도 너무 좋을거 같아요!',
 '홍삼이 여자몸에 안좋다고해서.. 그동안 안먹다가 꿀 스틱 보고 주문했습니다\n당떨어지기전에 먹으면 좋구요 밖에서도 쉽게 먹을수있어요. 이지컷이 잘 뜯어집니다.\n캡슐커피 내려먹을때 넣어먹을수도 있네요\n다먹으면 또 주문하려구요 ㅎㅎ',
 'BEST꿀이 피로 회복, 기관지에 좋다고 하는데 사실 어느정도가 적당한지, 너무 많이 먹으면 안좋은 건 아닌지 고민되더라구요ㅠ\n특히 부모님께 선물할 건데 꿀 많이 드셔서 당 수치가 올라갈까봐 걱정됬었는데 소량으로 먹기 좋게 스틱 포장 되어있어서 좋아요 !\n병에 담긴 꿀은 매일 꺼내서 떠먹기가 불편하고 적정량을 먹는건지 긴가민가했는데 부모님도 편하게 잘 드시고 계셔서 너무 만족스러운 선물이었습니다 💚💚💚',
 '한달사용기진짜 꿀이예요. 예전 시골에서 먹던 꿀맛과 같아요. 술먹고 먹어도 숙취에도 좋아요. 또 구매할거예요.',
 '너무좋아서 계속 재구매해요. 한포씩되어있어서 먹기편리해요  다먹으면 또구매 할예정입니다',
 '입 심심할때 간식으로 먹으려고 샀어요. 건강도 생각해서요ㅎㅎ. 벌꿀에 여러 영양소?가 많다고 하더라고요. 찔끔찔끔 먹는걸 좋아해서 간식으로도 괜찮아요. 포장도 깔끔하고 맛도있습니다. 일부러 3통 모두 여러가지 섞인걸로 샀습니다. 그때그때 먹고싶은거 골라먹는 재미가 있어요',
 '한달사용기재구매꿀의 점도도 딱!  좋쿠요. 일단 비닐 포장이 꿀을 빼서 먹을때 이빨 끝에 그 소름끼찌는 그런 느낌없이 쭉 잘 빠져 나와서 먹기 아주 좋아요.  커팅 부분도 잘 되어 있어서 아이도

In [4]:
# 한글 외의 모든 글자를 제거한다.
# 한글 정규식을 이용하여 한글 외의 모든 글자를 제거한다.

import re

def text_clearing(text):
    if isinstance(text, str):  # 값이 문자열인 경우에만 처리
        hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
        result = hangul.sub('', text)
        return result
    else:
        return ""


In [5]:
naver_shopping["ko_review"] = naver_shopping["리뷰"].apply(lambda x : text_clearing(x))

### bareunpy 예시

In [6]:
#bareunpy 예시

#import bareunpy as brn
#tagger = brn.Tagger(apikey="kpba-YOURKEY", domain="custom")
#>>> print(tagger.morphs('안녕하세요, 반가워요.'))
#['안녕', '하', '시', '어요', ',', '반갑', '어요', '.']
#>>> print(tagger.nouns('나비 허리에 새파란 초생달이 시리다.'))
#['나비', '허리', '초생달']
#>>> print(tagger.pos('햇빛이 선명하게 나뭇잎을 핥고 있었다.'))
#[('햇빛', 'NNG'), ('이', 'JKS'), ('선명', 'NNG'), ('하', 'XSA'), ('게', 'EC'), ('나뭇잎', 'NNG'),
    #('을', 'JKO'), ('핥', 'VV'), ('고', 'EC'), ('있', 'VX'), ('었', 'EP'), ('다', 'EF'), ('.', 'SF')]

In [7]:
tagger = brn.Tagger(apikey="koba-YGC27UA-OD3UPJY-VOIHT3I-G6IKLNA", domain="custom")

### bareunpy 토큰화

In [8]:
# bareunpy 라이브러리로 텍스트 데이터에서 형태소를 추출.
def get_pos (x) :
    pos = tagger.pos(x)
    
    # 단어와 품사를 합쳐서 하나의 단어로 만들어준다.
    result = []
    
    # 형태소의 수만큼 반복한다.
    # 조사인 것과 명사인 것이 같을 수 있기 때문에 구분해준다.
    # 형태소 벡터를 만들때 추후 사용
    for a1 in pos :
        result.append(f'{a1[0]}/{a1[1]}')
    
    return result

In [9]:
print(tagger.morphs('안녕하세요, 반가워요. 내 이름은 신익환'))
print(tagger.nouns('안녕하세요, 반가워요. 내 이름은 신익환'))
print(tagger.pos('안녕하세요, 반가워요. 내 이름은 신익환'))

['안녕하', '시', '어요', ',', '반갑', '어요', '.', '나', '의', '이름', '은', '신익환']
['나', '이름', '신익환']
[('안녕하', 'VA'), ('시', 'EP'), ('어요', 'EF'), (',', 'SP'), ('반갑', 'VA'), ('어요', 'EF'), ('.', 'SF'), ('나', 'NP'), ('의', 'JKG'), ('이름', 'NNG'), ('은', 'JX'), ('신익환', 'NNP')]


In [10]:
naver_shopping['ko_review']

0        한달사용기재구매먹기 편하고 품질도 좋아요 받는 손님도 좋아 하시고 구매등급도 따로 ...
1        간편하게 하루한포씩먹게되요냉장이나 냉동으로 해서 시원하게 먹으니 맛잇어요밤꿀이 많이...
2        홍삼이 여자몸에 안좋다고해서 그동안 안먹다가 꿀 스틱 보고 주문했습니다당떨어지기전에...
3        꿀이 피로 회복 기관지에 좋다고 하는데 사실 어느정도가 적당한지 너무 많이 먹으면 ...
4        한달사용기진짜 꿀이예요 예전 시골에서 먹던 꿀맛과 같아요 술먹고 먹어도 숙취에도 좋...
                               ...                        
57136                           선물했는데 잘 받았다고하네요 빠른 배송 감사해요
57137                                 설 선물로 준비했는데 좋은 것 같아요
57138                                     선물용으로 고급지고 좋습니다 
57139                                      고급스럽고 선물용으로 좋아요
57140                                            선물로 좋습니다 
Name: ko_review, Length: 57141, dtype: object

In [11]:
tagger.pos(naver_shopping['ko_review'][0])

[('한', 'XPN'),
 ('달', 'NNG'),
 ('사용', 'NNG'),
 ('기재', 'NNG'),
 ('구매', 'NNG'),
 ('먹', 'VV'),
 ('기', 'ETN'),
 ('편하', 'VA'),
 ('고', 'EC'),
 ('품질', 'NNG'),
 ('도', 'JX'),
 ('좋', 'VA'),
 ('아요', 'EF'),
 ('받', 'VV'),
 ('는', 'ETM'),
 ('손', 'NNG'),
 ('님', 'XSN'),
 ('도', 'JX'),
 ('좋아하', 'VV'),
 ('시', 'EP'),
 ('고', 'EC'),
 ('구매', 'NNG'),
 ('등급', 'NNG'),
 ('도', 'JX'),
 ('따로', 'MAG'),
 ('있', 'VA'),
 ('어서', 'EC'),
 ('좋', 'VA'),
 ('아요', 'EF'),
 ('문제', 'NNG'),
 ('생기', 'VV'),
 ('면', 'EC'),
 ('바로', 'MAG'),
 ('처리', 'NNG'),
 ('하', 'VV'),
 ('아', 'EC'),
 ('주', 'VX'),
 ('시', 'EP'),
 ('고', 'EC'),
 ('좋', 'VA'),
 ('아요', 'EF')]

In [21]:
# 'token'이라는 새로운 컬럼 추가 및 값 할당
naver_shopping['token'] = naver_shopping['ko_review'].apply(lambda x: tagger.pos(x))

# 결과 확인
print(naver_shopping['token'])

OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.


In [23]:
naver_shopping.head()

In [12]:
naver_shopping["ko_review"].tolist()

['한달사용기재구매먹기 편하고 품질도 좋아요 받는 손님도 좋아 하시고 구매등급도 따로 있어서 좋아요 문제 생기면 바로 처리 해주시고 좋아요',
 '간편하게 하루한포씩먹게되요냉장이나 냉동으로 해서 시원하게 먹으니 맛잇어요밤꿀이 많이 쓸줄알앗는데 쓴건 잘 안먹는편이기도 하구요  생각보다 진하니 먹을만했어요 요즘 면역이 많이 떨어지기도 했고 겸사겸사 구매했는데 마음에들어서 다음에 재구매해야겠어요 선물용으로도 너무 좋을거 같아요',
 '홍삼이 여자몸에 안좋다고해서 그동안 안먹다가 꿀 스틱 보고 주문했습니다당떨어지기전에 먹으면 좋구요 밖에서도 쉽게 먹을수있어요 이지컷이 잘 뜯어집니다캡슐커피 내려먹을때 넣어먹을수도 있네요다먹으면 또 주문하려구요 ㅎㅎ',
 '꿀이 피로 회복 기관지에 좋다고 하는데 사실 어느정도가 적당한지 너무 많이 먹으면 안좋은 건 아닌지 고민되더라구요ㅠ특히 부모님께 선물할 건데 꿀 많이 드셔서 당 수치가 올라갈까봐 걱정됬었는데 소량으로 먹기 좋게 스틱 포장 되어있어서 좋아요 병에 담긴 꿀은 매일 꺼내서 떠먹기가 불편하고 적정량을 먹는건지 긴가민가했는데 부모님도 편하게 잘 드시고 계셔서 너무 만족스러운 선물이었습니다 ',
 '한달사용기진짜 꿀이예요 예전 시골에서 먹던 꿀맛과 같아요 술먹고 먹어도 숙취에도 좋아요 또 구매할거예요',
 '너무좋아서 계속 재구매해요 한포씩되어있어서 먹기편리해요  다먹으면 또구매 할예정입니다',
 '입 심심할때 간식으로 먹으려고 샀어요 건강도 생각해서요ㅎㅎ 벌꿀에 여러 영양소가 많다고 하더라고요 찔끔찔끔 먹는걸 좋아해서 간식으로도 괜찮아요 포장도 깔끔하고 맛도있습니다 일부러 통 모두 여러가지 섞인걸로 샀습니다 그때그때 먹고싶은거 골라먹는 재미가 있어요',
 '한달사용기재구매꿀의 점도도 딱  좋쿠요 일단 비닐 포장이 꿀을 빼서 먹을때 이빨 끝에 그 소름끼찌는 그런 느낌없이 쭉 잘 빠져 나와서 먹기 아주 좋아요  커팅 부분도 잘 되어 있어서 아이도 잘 빼서 먹더라구요',
 '올인원으로 구입했는데 같은 꿀이여도 맛이 조금씩 달라서 신

In [14]:
# 형태소 벡터를 생성하기 위한 라이브러리
from sklearn.feature_extraction.text import CountVectorizer
# 형태소 벡터를 학습 벡터로 변환한다.
from sklearn.feature_extraction.text import TfidfTransformer

# 형태소 추출
index_vectorizer = CountVectorizer(tokenizer=lambda x : get_pos(x))
X = index_vectorizer.fit_transform(naver_shopping["ko_review"].tolist())

OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.
OOPS, no sentences.


In [15]:
index_vectorizer.vocabulary_

{'한/XPN': 18475,
 '달/NNG': 4770,
 '사용/NNG': 8976,
 '기재/NNG': 2868,
 '구매/NNG': 1886,
 '먹/VV': 6965,
 '기/ETN': 2650,
 '편하/VA': 17847,
 '고/EC': 1476,
 '품질/NNG': 18176,
 '도/JX': 5286,
 '좋/VA': 15508,
 '아요/EF': 11229,
 '받/VV': 7773,
 '는/ETM': 4405,
 '손/NNG': 10048,
 '님/XSN': 4531,
 '좋아하/VV': 15538,
 '시/EP': 10538,
 '등급/NNG': 5721,
 '따로/MAG': 5791,
 '있/VA': 14386,
 '어서/EC': 11814,
 '문제/NNG': 7411,
 '생기/VV': 9573,
 '면/EC': 7099,
 '바로/MAG': 7655,
 '처리/NNG': 16551,
 '하/VV': 18328,
 '아/EC': 11073,
 '주/VX': 15603,
 '간편하/VA': 766,
 '게/EC': 1333,
 '하루/NNG': 18386,
 '한/MMN': 18469,
 '포/NNG': 17894,
 '씩/XSN': 11065,
 '되/VV': 5450,
 '요/EC': 12831,
 '냉장/NNG': 3955,
 '이나/JC': 13946,
 '냉동/NNG': 3947,
 '으로/JKB': 13644,
 '아서/EC': 11185,
 '시원하/VA': 10612,
 '으니/EC': 13600,
 '맛잇/VV': 6742,
 '어요/EF': 11858,
 '밤꿀/NNG': 7821,
 '이/JKS': 13909,
 '많이/MAG': 6609,
 '쓸줄알/NNG': 11021,
 '앗/VV': 11520,
 '는데/EC': 4440,
 '쓰/VV': 10986,
 'ㄴ거/EC': 22,
 'ㄴ/JX': 11,
 '잘/MAG': 14561,
 '안/MAG': 11386,
 '편/NNB': 17811,
 '이/VCP': 

In [16]:
# CountVectorizer로 생성한 형태소 벡터에서 단어 빈도수를 계산
word_frequencies = X.sum(axis=0)

# 각 단어와 빈도수를 매핑하는 딕셔너리 생성
word_frequency_dict = {}
for word, index in index_vectorizer.vocabulary_.items():
    word_frequency_dict[word] = word_frequencies[0, index]

# 빈도수가 큰 순서대로 정렬된 딕셔너리 생성
sorted_word_frequency_dict = dict(sorted(word_frequency_dict.items(), key=lambda item: item[1], reverse=True))

# 결과 출력
for word, frequency in sorted_word_frequency_dict.items():
    print(f"{word}: {frequency}")


고/EC: 41755
좋/VA: 37492
먹/VV: 36799
도/JX: 30821
어요/EF: 25665
아요/EF: 24535
았/EP: 23877
요/JX: 23402
어/EC: 19746
에/JKB: 19252
이/VCP: 18980
기/ETN: 18624
이/JKS: 17904
네/EF: 14890
ㄴ/ETM: 14063
선물/NNG: 13163
아서/EC: 13130
ㅂ니다/EF: 12929
는데/EC: 12828
꿀/NNG: 12400
한/MMN: 12133
습니다/EF: 11550
시/EP: 11491
아/EC: 11408
어서/EC: 11302
게/EC: 10923
었/EP: 10639
으로/JKB: 10271
너무/MAG: 10156
포장/NNG: 9174
편하/VA: 9107
잘/MAG: 8995
맛있/VA: 8970
맛/NNG: 8679
구매/NNG: 8624
는/ETM: 8435
ㄹ/ETM: 8320
있/VA: 8129
달/NNB: 8080
있/VX: 7741
로/JKB: 7659
거/NNB: 7362
하/VV: 7319
보/VX: 7069
용/XSN: 7048
재/XPN: 7039
사용기/NNG: 6956
배송/NNG: 6663
가/JKS: 6600
ㅎ/NNG: 6586
같/VA: 6368
구매하/VV: 5980
을/ETM: 5928
은/ETM: 5571
빠르/VA: 5525
씩/XSN: 5038
은/JX: 4597
주/VX: 4578
되/VV: 4555
하/XSV: 4480
을/JKO: 4478
것/NNB: 4448
사/VV: 4419
들/XSN: 4410
라/EC: 4397
때/NNG: 4362
주문하/VV: 4239
좋아하/VV: 4189
간편하/VA: 4099
수/NNB: 4017
받/VV: 3750
들/VV: 3707
선물하/VV: 3657
스틱/NNG: 3613
하/VX: 3558
겠/EP: 3531
는/JX: 3501
사용/NNG: 3253
감사하/VV: 3184
ㄴ/JX: 3182
또/MAG: 3158
다/MAG: 29

In [18]:
# 딕셔너리를 DataFrame으로 변환합니다.
df = pd.DataFrame(list(sorted_word_frequency_dict.items()), columns=['Top Nouns', '빈도수'])

# DataFrame을 출력합니다.
df.to_csv("naver_shopping_frequent.csv")

In [31]:
result = []

for idx, value in df['Top Nouns'].iteritems():
    word = value.split('/')[0]
    pos = value.split('/')[1]
    result.append([word, pos])

df_ns = pd.DataFrame(result, columns=['단어', '품사'])

# 결과 출력
df_ns

C:\Users\NT550\AppData\Local\Temp\ipykernel_1912\2496072620.py:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, value in df['Top Nouns'].iteritems():


,단어,품사
0,고,EC
1,좋,VA
2,먹,VV
3,도,JX
4,어요,EF
...,...,...
19498,숙훅숙후숙,NNG
19499,휴구매,NNG
19500,묘즘,NNG
19501,난리브루스,MAG


### 블로그 토큰화

In [20]:
naver_blog = pd.read_csv(r'C:\Users\NT550\asac\SIH\hsw\finaldata\naver_blog_text_scaling_last_tokens.csv')
naver_blog.head()

,개시일,주소,꿀or허니,찐주소,내용,변환된 내용,tokens,tokens_all,tokens_nouns,tokens_verbs,tokens_adjectives
0,2023.08.17.,https://blog.naver.com/stellak_kim/223185433857,0,https://blog.naver.com/PostView.naver?blogId=s...,"['올리 꿀스틱\n총 30포 (3종세트) 22,000원', '올리 꿀스틱\n총 30...",올리 꿀스틱 총 30포 3종세트 22000원 올리 꿀스틱 총 30포 3종세트 220...,"['올리', '꿀', '스틱', '총', '30', '포', '3', '종', '세...","['올리', '꿀', '스틱', '총', '30', '포', '3', '종', '세...","['꿀', '스틱', '포', '종', '세트', '꿀', '스틱', '포', '종...","['올리', '올리', '드리', '찾', '먹', '올리', '맛보', '열', ...","['크', '깊', '많', '있', '달콤하', '어떻', '빠르', '이렇', ..."
1,2023.08.10.,https://blog.naver.com/lucy_bear/223181233068,0,https://blog.naver.com/PostView.naver?blogId=l...,"['사실 부모님이나 할머니, 선생님 등\n어른 분들께 선물할 때가\n또래 친구들에서...",사실 부모님이나 할머니 선생님 등 어른 분들께 선물할 때가 또래 친구들에서 선물할 ...,"['사실', '부모', '님', '이나', '할머니', '선생', '님', '등',...","['사실', '부모', '님', '이나', '할머니', '선생', '님', '어른'...","['부모', '할머니', '선생', '어른', '선물', '또래', '친구', '선...","['싫어하', '좋아하', '들', '싫어하', '올리', '좋아하', '사', '...","['좋', '좋', '차분하', '있', '좋', '좋', '좋', '예쁘', '다..."
2,2023.08.07.,https://blog.naver.com/njellsol1/223176885972,0,https://blog.naver.com/PostView.naver?blogId=n...,"[',,,,,,,아임힐링 엔젯허니 마누카꿀 스틱 효능 / 마누카허니 핑크벌 확인하고...",아임힐링 엔젯허니 마누카꿀 스틱 효능 마누카허니 핑크벌 확인하고 선택하세요아임힐링...,"['아이', 'ᆷ', '힐링', '엔젯허니', '마누카꿀', '스틱', '효능', ...","['아이', 'ᆷ', '힐링', '엔젯허니', '마누카꿀', '스틱', '효능', ...","['힐링', '엔젯허니', '마누카꿀', '스틱', '효능', '마누카허니', '핑...","['챙기', '쏘', '타', '마시', '닿', '먹', '즐기', '살', '주...","['간편하', '귀하', '있', '좋', '좋', '편하', '좋', '진하', ..."
3,2023.07.15.,https://blog.naver.com/she_op/223156627263,0,https://blog.naver.com/PostView.naver?blogId=s...,['7월 25일\n내 첫 조카가 태어난 날.\n처음 안아봤던 그 작은 아기의 따뜻했...,7월 25일 내 첫 조카가 태어난 날. 처음 안아봤던 그 작은 아기의 따뜻했던 그 ...,"['7', '월', '25', '일', '나', '의', '첫', '조카', '가'...","['7', '25', '첫', '조카', '태어나', '날', '처음', '안', ...","['조카', '날', '아기', '느낌', '시간', '동안', '언니', '오빠'...","['태어나', '안', '잊', '오가', '치', '들', '원하', '드리', ...","['작', '많', '그렇', '그렇', '빠르', '좋', '있', '뒤늦', '..."
4,2021.12.01.,https://blog.naver.com/sumika1207/222583012077,0,https://blog.naver.com/PostView.naver?blogId=s...,"[',,,,,,,선물용 스틱꿀, 답례품으로 좋아요!,우아하게 즐기는 스틱꿀,날이 점...",선물용 스틱꿀 답례품으로 좋아요우아하게 즐기는 스틱꿀날이 점점 추워지니 따뜻한 차가...,"['선물', '용', '스틱', '꿀', '답례', '품', '으로', '좋', '...","['선물', '용', '스틱', '꿀', '답례', '품', '좋', '어요', '...","['선물', '스틱', '꿀', '답례', '품', '스틱', '꿀', '날', '...","['즐기', '생각나', '생기', '나오', '먹', '맡', '먹', '담기',...","['좋', '이렇', '좋', '좋', '있', '아담하', '상큼하', '좋', ..."


In [21]:
naver_blog["str_content"] = naver_blog["변환된 내용"].apply(lambda x : text_clearing(x))

In [22]:
# 'token'이라는 새로운 컬럼 추가 및 값 할당
naver_blog['token'] = naver_blog['str_content'].apply(lambda x: tagger.pos(x))

# 결과 확인
naver_blog['token']

KeyboardInterrupt: 

In [ ]:
# 형태소 추출
index_vectorizer = CountVectorizer(tokenizer=lambda x : get_pos(x))
X = index_vectorizer.fit_transform(naver_blog["str_content"].tolist())
X

c:\ProgramData\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<1826x40644 sparse matrix of type '<class 'numpy.int64'>'
	with 559308 stored elements in Compressed Sparse Row format>

In [ ]:
index_vectorizer.vocabulary_

{'올리/NNG': 25135,
 '꿀스틱총/NNG': 5631,
 '포/NNG': 36552,
 '종세트/NR': 30992,
 '원/NNB': 26918,
 '종세트/NNG': 30990,
 '원다음/NNG': 26944,
 '달/NNG': 8650,
 '이/VCP': 28071,
 '면/EC': 13701,
 '민족/NNG': 14658,
 '대/XPN': 9061,
 '명절/NNG': 13782,
 'ㄴ/ETM': 6,
 '추석/NNG': 33843,
 '이/JKS': 28062,
 '있/VA': 29164,
 '는데/EC': 7745,
 '요/JX': 25443,
 '한국인/NNG': 37998,
 '에게/JKB': 23864,
 '크/VA': 34949,
 '의미/NNG': 28013,
 '가/JKS': 616,
 '는/ETM': 7697,
 '만큼/JKB': 12974,
 '소중하/VA': 19866,
 '분/NNB': 16933,
 '드리/VV': 10267,
 'ㄹ/ETM': 185,
 '만/NNB': 12883,
 '하/XSA': 37669,
 '뜻/NNG': 11282,
 '깊/VA': 4407,
 '은/ETM': 27748,
 '선물/NNG': 19184,
 '을/JKO': 27839,
 '찾/VV': 32940,
 '들/XSN': 10336,
 '많/VA': 12987,
 '을/ETM': 27838,
 '것/NNB': 1945,
 '같/VA': 1349,
 '아요/EF': 22299,
 '가격대/NNG': 630,
 '부담스럽/VA': 16784,
 '지/EC': 31682,
 '않/VX': 22765,
 '으면서/EC': 27710,
 '도/JX': 9584,
 '쉽/VA': 20487,
 '게/EC': 1984,
 '먹/VV': 13485,
 '수/NNB': 20136,
 '달콤하/VA': 8790,
 '꿀/NNG': 4826,
 '은/JX': 27749,
 '어떻/VA': 23444,
 '시/EP': 21025,
 '어요/EF': 

In [ ]:
# CountVectorizer로 생성한 형태소 벡터에서 단어 빈도수를 계산
word_frequencies = X.sum(axis=0)

# 각 단어와 빈도수를 매핑하는 딕셔너리 생성
word_frequency_dict = {}
for word, index in index_vectorizer.vocabulary_.items():
    word_frequency_dict[word] = word_frequencies[0, index]

# 빈도수가 큰 순서대로 정렬된 딕셔너리 생성
sorted_word_frequency_dict = dict(sorted(word_frequency_dict.items(), key=lambda item: item[1], reverse=True))

# 결과 출력
for word, frequency in sorted_word_frequency_dict.items():
    print(f"{word}: {frequency}")

꿀/NNG: 37223
ㄴ/ETM: 36535
에/JKB: 32254
이/JKS: 25576
고/EC: 25565
이/VCP: 25162
어/EC: 24653
는/ETM: 23934
을/JKO: 22731
도/JX: 20272
먹/VV: 17528
아/EC: 17518
좋/VA: 16543
게/EC: 15232
의/JKG: 14754
은/JX: 13857
있/VA: 13688
요/JX: 13314
ㄹ/ETM: 13187
기/ETN: 12827
으로/JKB: 12025
가/JKS: 11699
어요/EF: 11295
은/ETM: 10875
는/JX: 10866
로/JKB: 10104
하/VV: 9981
있/VX: 9766
시/EP: 9585
스틱/NNG: 9142
선물/NNG: 9027
꿀스틱/NNG: 8593
를/JKO: 8399
았/EP: 8340
수/NNB: 8200
었/EP: 7649
에서/JKB: 7574
어서/EC: 7545
아서/EC: 7353
들/XSN: 6853
되/VV: 6739
는데/EC: 6737
을/ETM: 6646
아요/EF: 6592
ㅂ니다/EF: 6575
천연/NNG: 6153
보/VX: 6018
거/NNB: 5796
들/VV: 5642
허니스틱/NNG: 5579
면/EC: 5527
때/NNG: 5477
것/NNB: 5188
니/EC: 5136
아카시아/NNG: 5028
같/VA: 4788
한/MMN: 4671
과/JC: 4555
벌꿀/NNG: 4483
지/EC: 4471
주/VX: 4464
맛/NNG: 4438
하/VX: 4173
ㅎ/NNG: 4163
다/EF: 3957
마누카/NNG: 3858
ㅋ/NNG: 3826
제품/NNG: 3766
않/VX: 3624
맛있/VA: 3584
습니다/EF: 3342
만들/VV: 3304
라/EC: 3294
만/JX: 3291
하/XSV: 3261
넣/VV: 3225
간편하/VA: 3198
챙기/VV: 3185
건강/NNG: 3157
너무/MAG: 3088
세트/NNG: 3006
포/NNG: 296

In [27]:
# 딕셔너리를 DataFrame으로 변환합니다.
blog = pd.DataFrame(list(sorted_word_frequency_dict.items()), columns=['토큰', '빈도수'])

# DataFrame을 출력합니다.
blog.to_csv('naver_blog.frequent.csv')

In [28]:
result = []

for idx, value in blog['토큰'].iteritems():
    word = value.split('/')[0]
    pos = value.split('/')[1]
    result.append([word, pos])

df_blog = pd.DataFrame(result, columns=['단어', '품사'])

# 결과 출력
df_blog

C:\Users\NT550\AppData\Local\Temp\ipykernel_6496\173774057.py:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, value in blog['토큰'].iteritems():


,단어,품사
0,고,EC
1,좋,VA
2,먹,VV
3,도,JX
4,어요,EF
...,...,...
19498,숙훅숙후숙,NNG
19499,휴구매,NNG
19500,묘즘,NNG
19501,난리브루스,MAG


In [46]:
naver_blog = pd.read_csv(r'C:\Users\NT550\asac\SIH\SIH\code2\result\naver_shopping_tokenizer.csv')
naver_blog.head()

,Unnamed: 0.1,Unnamed: 0,제품명,가격,평균평점,아이디,평점,날짜,종류,요약,리뷰,ko_review,token
0,0,0,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,miri*******,5.0,23.08.16.,30포 선택1: 아카시아 / 30포 선택2: 아카시아 / 30포 선택3: 아카시아\n,유통기한아주 넉넉해요포장꼼꼼해요편리편리해요,한달사용기재구매먹기 편하고 품질도 좋아요. 받는 손님도 좋아 하시고 구매등급도 따로...,한달사용기재구매먹기 편하고 품질도 좋아요 받는 손님도 좋아 하시고 구매등급도 따로 ...,"[('한', 'XPN'), ('달', 'NNG'), ('사용', 'NNG'), ('..."
1,1,1,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,dltm*****,5.0,23.07.28.,30포 선택1: 밤 / 30포 선택2: 밤 / 30포 선택3: 밤\n,유통기한꽤 남았어요포장꼼꼼해요편리편리해요,간편하게 하루한포씩먹게되요~냉장이나 냉동으로 해서 시원하게 먹으니 맛잇어요!!밤꿀이...,간편하게 하루한포씩먹게되요냉장이나 냉동으로 해서 시원하게 먹으니 맛잇어요밤꿀이 많이...,"[('간편하', 'VA'), ('게', 'EC'), ('하루', 'NNG'), ('..."
2,2,2,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,yjdr****,5.0,23.08.06.,30포 선택1: 올인원 / 30포 선택2: 올인원 / 30포 선택3: 아카시아\n,유통기한꽤 남았어요포장꼼꼼해요편리편리해요,홍삼이 여자몸에 안좋다고해서.. 그동안 안먹다가 꿀 스틱 보고 주문했습니다\n당떨어...,홍삼이 여자몸에 안좋다고해서 그동안 안먹다가 꿀 스틱 보고 주문했습니다당떨어지기전에...,"[('홍삼', 'NNG'), ('이', 'JKS'), ('여자', 'NNG'), (..."
3,3,3,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,2133***,5.0,23.07.13.,30포 선택1: 밤 / 30포 선택2: 야생화 / 30포 선택3: 올인원\n,유통기한아주 넉넉해요포장꼼꼼해요편리편리해요,"BEST꿀이 피로 회복, 기관지에 좋다고 하는데 사실 어느정도가 적당한지, 너무 많...",꿀이 피로 회복 기관지에 좋다고 하는데 사실 어느정도가 적당한지 너무 많이 먹으면 ...,"[('꿀', 'NNG'), ('이', 'JKS'), ('피로', 'NNG'), ('..."
4,4,4,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,klys****,5.0,23.08.12.,30포 선택1: 야생화 / 30포 선택2: 밤 / 30포 선택3: 아카시아\n,유통기한아주 넉넉해요포장꼼꼼해요편리편리해요,한달사용기진짜 꿀이예요. 예전 시골에서 먹던 꿀맛과 같아요. 술먹고 먹어도 숙취에도...,한달사용기진짜 꿀이예요 예전 시골에서 먹던 꿀맛과 같아요 술먹고 먹어도 숙취에도 좋...,"[('한', 'MMN'), ('달', 'NNB'), ('사용기', 'NNG'), (..."


In [47]:
# 명사, 동사, 형용사 품사 태그 정의
nouns_tags = ['NNG','NNP'] # NNG, NNP, 
verbs_tags = ['VV'] # VV
adjectives_tags = ['VA'] # VA

# 각 품사를 추출하여 열 추가
naver_blog['nouns'] = naver_blog['token'].apply(lambda x: [word for word, tag in x if tag in nouns_tags])
naver_blog['verbs'] = naver_blog['token'].apply(lambda x: [word for word, tag in x if tag in verbs_tags])
naver_blog['adjectives'] = naver_blog['token'].apply(lambda x: [word for word, tag in x if tag in adjectives_tags])

# 결과 출력
print(naver_blog[['nouns', 'verbs', 'adjectives']])

ValueError: not enough values to unpack (expected 2, got 1)

In [44]:
df_blog[df_blog['품사'] == 'VA']

,단어,품사
1,좋,VA
30,편하,VA
32,맛있,VA
37,있,VA
50,같,VA
...,...,...
19411,황홀하,VA
19437,맛중,VA
19459,날씬하,VA
19467,원활하,VA


### 형태소 분석기 비교
- bareun VS kiwi


In [23]:
bareun = pd.read_csv(r"C:\Users\NT550\asac\SIH\SIH\code2\result\naver_blog_tokenizer.csv")
kiwi = pd.read_csv(r"C:\Users\NT550\asac\SIH\hsw\finaldata\naver_blog_text_scaling_last_tokens.csv")

In [24]:
bareun

,Unnamed: 0,개시일,주소,꿀or허니,찐주소,내용,변환된 내용,str_content,token
0,0,2023.08.17.,https://blog.naver.com/stellak_kim/223185433857,0,https://blog.naver.com/PostView.naver?blogId=s...,"['올리 꿀스틱\n총 30포 (3종세트) 22,000원', '올리 꿀스틱\n총 30...",올리 꿀스틱n총 30포 3종세트 22000원 올리 꿀스틱n총 30포 3종세트 220...,올리 꿀스틱총 포 종세트 원 올리 꿀스틱총 포 종세트 원다음 달이면 민족 대명절인 ...,"[('올리', 'NNG'), ('꿀스틱총', 'NNG'), ('포', 'NNG'),..."
1,1,2023.08.10.,https://blog.naver.com/lucy_bear/223181233068,0,https://blog.naver.com/PostView.naver?blogId=l...,"['사실 부모님이나 할머니, 선생님 등\n어른 분들께 선물할 때가\n또래 친구들에서...",사실 부모님이나 할머니 선생님 등n어른 분들께 선물할 때가n또래 친구들에서 선물할 ...,사실 부모님이나 할머니 선생님 등어른 분들께 선물할 때가또래 친구들에서 선물할 때보...,"[('사실', 'MAG'), ('부모', 'NNG'), ('님', 'XSN'), (..."
2,2,2023.08.07.,https://blog.naver.com/njellsol1/223176885972,0,https://blog.naver.com/PostView.naver?blogId=n...,"[',,,,,,,아임힐링 엔젯허니 마누카꿀 스틱 효능 / 마누카허니 핑크벌 확인하고...",아임힐링 엔젯허니 마누카꿀 스틱 효능 마누카허니 핑크벌 확인하고 선택하세요아임힐링...,아임힐링 엔젯허니 마누카꿀 스틱 효능 마누카허니 핑크벌 확인하고 선택하세요아임힐링...,"[('아임힐링', 'NNG'), ('엔젯허니', 'NNG'), ('마누카', 'NN..."
3,3,2023.07.15.,https://blog.naver.com/she_op/223156627263,0,https://blog.naver.com/PostView.naver?blogId=s...,['7월 25일\n내 첫 조카가 태어난 날.\n처음 안아봤던 그 작은 아기의 따뜻했...,7월 25일n내 첫 조카가 태어난 날.n처음 안아봤던 그 작은 아기의 따뜻했던n그 ...,월 일내 첫 조카가 태어난 날처음 안아봤던 그 작은 아기의 따뜻했던그 느낌을 아직도...,"[('월', 'NNG'), ('일', 'NNG'), ('내', 'NNB'), ('첫..."
4,4,2021.12.01.,https://blog.naver.com/sumika1207/222583012077,0,https://blog.naver.com/PostView.naver?blogId=s...,"[',,,,,,,선물용 스틱꿀, 답례품으로 좋아요!,우아하게 즐기는 스틱꿀,날이 점...",선물용 스틱꿀 답례품으로 좋아요우아하게 즐기는 스틱꿀날이 점점 추워지니 따뜻한 차가...,선물용 스틱꿀 답례품으로 좋아요우아하게 즐기는 스틱꿀날이 점점 추워지니 따뜻한 차가...,"[('선물', 'NNG'), ('용', 'XSN'), ('스틱', 'NNG'), (..."
...,...,...,...,...,...,...,...,...,...
1821,1821,2019.02.20.,https://blog.naver.com/chulwoo84/221470432922,1,https://blog.naver.com/PostView.naver?blogId=c...,"[',,,,,,,자기소개를 입력하세요.,프로필,,^,,◀ 2023.08 ▶,월별보기...",자기소개를 입력하세요.프로필 2023.08 월별보기2019. 2. 20. 1423h...,자기소개를 입력하세요프로필 월별보기 안녕하세요 잇님들 영미쨩이예요오늘은 달콤함...,"[('자기소개', 'NNG'), ('를', 'JKO'), ('입력하', 'VV'),..."
1822,1822,2022.08.22.,https://blog.naver.com/lela1001/222854787520,1,https://blog.naver.com/PostView.naver?blogId=l...,"[',,,,,,,스윗홈파티 추석상품 오픈/ 명절 선생님 선물, 명절 부모님 선물, ...",스윗홈파티 추석상품 오픈 명절 선생님 선물 명절 부모님 선물 명절 구디백 명절 답례...,스윗홈파티 추석상품 오픈 명절 선생님 선물 명절 부모님 선물 명절 구디백 명절 답례...,"[('스윗홈파티', 'NNG'), ('추석', 'NNG'), ('상품', 'NNG'..."
1823,1823,2021.05.23.,https://blog.naver.com/saeng43/222362770703,1,https://blog.naver.com/PostView.naver?blogId=s...,"['', '안녕하세요! 마느리에용\n요즘 꾸덕한 요거트가 땡기는데\n만들어 먹기엔 ...",안녕하세요 마느리에용n요즘 꾸덕한 요거트가 땡기는데n만들어 먹기엔 너무 귀찮기도 ...,안녕하세요 마느리에용요즘 꾸덕한 요거트가 땡기는데만들어 먹기엔 너무 귀찮기도 하고...,"[('안녕하', 'VA'), ('시', 'EP'), ('어요', 'EF'), ('마..."
1824,1824,2021.02.22.,https://blog.naver.com/gustongue_tv/222252518165,1,https://blog.naver.com/PostView.naver?blogId=g...,"[',,,,,,,[뭐먹지?] 구스통 2월 넷째 주 스페셜 라인업,이제 정말 완연한 ...",뭐먹지 구스통 2월 넷째 주 스페셜 라인업이제 정말 완연한 봄날씨 같죠공원에 나가보...,뭐먹지 구스통 월 넷째 주 스페셜 라인업이제 정말 완연한 봄날씨 같죠공원에 나가보면...,"[('뭐', 'NP'), ('먹', 'VV'), ('지', 'EC'), ('구스통'..."


In [12]:
# 명사, 동사, 형용사 품사 태그 정의
nouns_tags = ['NNG', 'NP', 'NNP','NNB','NF','NA']
verbs_tags = ['VV','VX','VCP','VCN','NV']
adjectives_tags = ['VA']

# 각 품사를 추출하여 열 추가
naver_blog['nouns'] = naver_blog['token'].apply(lambda x: [word for word, tag in x if tag in nouns_tags])
naver_blog['verbs'] = naver_blog['token'].apply(lambda x: [word for word, tag in x if tag in verbs_tags])
naver_blog['adjectives'] = naver_blog['token'].apply(lambda x: [word for word, tag in x if tag in adjectives_tags])

# 결과 출력
print(naver_blog[['nouns', 'verbs', 'adjectives']])

C:\Users\NT550\AppData\Local\Temp\ipykernel_11004\3193411468.py:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, value in bareun['token'].iteritems():


IndexError: list index out of range

### 연관분석